In [1]:
import torch

from tqdm import tqdm
from IPython.display import clear_output

from facenet_pytorch import MTCNN, InceptionResnetV1

import math

import pathlib

import torchvision.transforms as transforms

In [2]:
# If required, create a face detection pipeline using MTCNN:
mtcnn = MTCNN(
    image_size=128, margin=0, min_face_size=20,
    thresholds=[0.6, 0.7, 0.7], factor=0.709, post_process=True,
    device=0
)

In [3]:
device="cuda:0"

In [4]:
IMAGE_EXTENSIONS = {'bmp', 'jpg', 'jpeg', 'pgm', 'png', 'ppm',
                    'tif', 'tiff', 'webp'}

In [5]:
# class ImagePathDataset(torch.utils.data.Dataset):
#     def __init__(self, files, transforms=None):
#         self.files = files
#         self.transforms = transforms

#     def __len__(self):
#         return len(self.files)

#     def __getitem__(self, i):
#         path = self.files[i]
#         img = Image.open(path).convert('RGB')
#         if self.transforms is not None:
#             img = self.transforms(img)
#         return img
from imageDatasetLoader import ImagePathDataset

tx = transforms.Compose([
#     transforms.Lambda(lambda t:t.convert("RGB")),
    transforms.Resize([128,128]),
    transforms.ToTensor(),
#     transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
    
])

batch_size = 1000

def make_dataloader(path_of_dataset):
    path = pathlib.Path(path_of_dataset)
    files = sorted([file for ext in IMAGE_EXTENSIONS
                           for file in path.glob('*.{}'.format(ext))])
    dataset = ImagePathDataset(files,transform= tx)
    dataloader = torch.utils.data.DataLoader(dataset,
                                    batch_size=batch_size,
                                    shuffle=False,
                                    drop_last=False,
                                    num_workers=1)
    return dataloader
    

In [6]:
synthetic_image_path = "D:/ddpm/generated_images/"
real_image_path = "D:/ddpm/real_images/"
syntheticdataloader=make_dataloader(synthetic_image_path)
realdataloader = make_dataloader(real_image_path)
next(iter(syntheticdataloader)).cuda().shape

torch.Size([1000, 3, 128, 128])

In [ ]:
fsd_scores=[]
fss_scores=[]
with torch.no_grad():
    for i in tqdm(range(1,10001)):
            clear_output(wait=True)
            img0=next(iter(syntheticdataloader)).cuda()
            img1=next(iter(realdataloader)).cuda()
            # Create an inception resnet (in eval mode):
            resnet = InceptionResnetV1(pretrained='vggface2').eval().cuda()
            img_embedding0 = resnet(img0)
            img_embedding1 = resnet(img1)
            fsd = torch.sum(torch.abs((img_embedding0-img_embedding1)))/batch_size
            print('fsd=',fsd)
            fsd_scores.append(fsd)
            cos = torch.nn.CosineSimilarity(dim=1)
            fss = torch.sum(cos(img_embedding0,img_embedding1))/batch_size
            print('fss=',fss)
            fss_scores.append(fss)
            del resnet

In [ ]:
distances=torch.tensor(fss_scores)
print(f"Total number of results: {distances.shape[0]}")
mean_distance=torch.mean(distances)
std_error=torch.std(distances)/math.sqrt(distances.shape[0])

print(f"fss score:{mean_distance}|Std Error:{std_error}")  

In [ ]:
distances=torch.tensor(fsd_scores)
print(f"Total number of results: {distances.shape[0]}")
mean_distance=torch.mean(distances)
std_error=torch.std(distances)/math.sqrt(distances.shape[0])

print(f"fsd score:{mean_distance}|Std Error:{std_error}")  